In [1]:
# requirements for this example:
%pip install -qq docling docling-core python-dotenv langchain-text-splitters langchain-huggingface langchain-milvus

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import warnings

warnings.filterwarnings(action="ignore", category=UserWarning, module="pydantic|torch")
warnings.filterwarnings(action="ignore", category=FutureWarning, module="easyocr")

## Setup

### Loader and splitter

Below we set up:
- a `Loader` which will be used to create LangChain documents, and
- a splitter, which will be used to split these documents

In [4]:
from enum import Enum
from typing import Iterator

from langchain_core.document_loaders import BaseLoader
from langchain_core.documents import Document as LCDocument
from pydantic import BaseModel

from docling.document_converter import DocumentConverter


class DocumentMetadata(BaseModel):
    dl_doc_hash: str
    # source: str


class DoclingPDFLoader(BaseLoader):
    class ParseType(str, Enum):
        MARKDOWN = "markdown"
        # JSON = "json"

    def __init__(self, file_path: str | list[str], parse_type: ParseType) -> None:
        self._file_paths = file_path if isinstance(file_path, list) else [file_path]
        self._parse_type = parse_type
        self._converter = DocumentConverter()

    def lazy_load(self) -> Iterator[LCDocument]:
        for source in self._file_paths:
            dl_doc = self._converter.convert_single(source).output
            match self._parse_type:
                case self.ParseType.MARKDOWN:
                    text = dl_doc.export_to_markdown()
                # case self.ParseType.JSON:
                #     text = dl_doc.model_dump_json()
                case _:
                    raise RuntimeError(
                        f"Unexpected parse type encountered: {self._parse_type}"
                    )
            lc_doc = LCDocument(
                page_content=text,
                metadata=DocumentMetadata(
                    dl_doc_hash=dl_doc.file_info.document_hash,
                ).model_dump(),
            )
            yield lc_doc

In [5]:
FILE_PATH = "https://arxiv.org/pdf/2206.01062"  # DocLayNet paper

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = DoclingPDFLoader(
    file_path=FILE_PATH,
    parse_type=DoclingPDFLoader.ParseType.MARKDOWN,
)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

We now used the above-defined objects to get the document splits:

In [7]:
docs = loader.load()
splits = text_splitter.split_documents(docs)

### Embeddings

In [8]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

HF_EMBED_MODEL_ID = "BAAI/bge-small-en-v1.5"
embeddings = HuggingFaceEmbeddings(model_name=HF_EMBED_MODEL_ID)

### Vector store

In [9]:
from tempfile import TemporaryDirectory

from langchain_milvus import Milvus

MILVUS_URI = os.environ.get(
    "MILVUS_URL", f"{(tmp_dir := TemporaryDirectory()).name}/milvus_demo.db"
)

vectorstore = Milvus.from_documents(
    splits,
    embeddings,
    connection_args={"uri": MILVUS_URI},
    drop_old=True,
)

### LLM

In [10]:
from langchain_huggingface import HuggingFaceEndpoint

HF_API_KEY = os.environ.get("HF_API_KEY")
HF_LLM_MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.3"

llm = HuggingFaceEndpoint(
    repo_id=HF_LLM_MODEL_ID,
    huggingfacehub_api_token=HF_API_KEY,
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/pva/.cache/huggingface/token
Login successful


## RAG

In [11]:
from typing import Iterable

from langchain_core.documents import Document as LCDocument
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs: Iterable[LCDocument]):
    return "\n\n".join(doc.page_content for doc in docs)


retriever = vectorstore.as_retriever()

prompt = PromptTemplate.from_template(
    "Context information is below.\n---------------------\n{context}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {question}\nAnswer:\n"
)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [12]:
rag_chain.invoke("How many pages were human annotated for DocLayNet?")

'The human annotation of DocLayNet was performed on 80863 pages.\n\nExplanation:\nThe information is found in the paragraph "DocLayNet contains 80863 PDF pages" in the context.'